In [1]:
import torch
import torch.nn as nn

## 演習
以下のいずれかの環境でGPUあり、なしを設定し,PytorchからGPUが見えるか確認してください.
- Google Colaboratory 
- Kaggle Kernel

資料にはあまり情報を載せていないので検索してみてください

In [2]:
print(torch.cuda.is_available())

False


## 演習2: 以下の微分をpytorchで計算してみてください.
__注意__: 入力は適当に決めてください.
- $e^{x}$
- $\cos x$
- $\sin x$
- $x^2$
- $\frac{e^{ax}}{e^{ax} + e^{bx}}$

In [6]:
X1 = torch.tensor(3., requires_grad=True)

In [7]:
print(torch.autograd.grad(torch.exp(X1), X1))
print(torch.autograd.grad(torch.cos(X1), X1))
print(torch.autograd.grad(torch.sin(X1), X1))
print(torch.autograd.grad(X1 ** 2, X1))
print(torch.autograd.grad(torch.exp(3 *X1)/(torch.exp(3*X1) + torch.exp(2*X1)), X1))

(tensor(20.0855),)
(tensor(-0.1411),)
(tensor(-0.9900),)
(tensor(6.),)
(tensor(0.0452),)


## 演習3: 以下の微分をpytorchで計算してみてください.
以降では,
$y = \frac{e^{ax}}{e^{ax} + e^{bx}}$として$z$を$x$で微分してください.
__注意__: 入力は適当に決めてください.
- $z = sin(y)$
- $\cos y$
- $\sin y$
- $y^2$
- $\frac{e^{ay}}{e^{ay} + e^{by}}$

In [8]:
y = torch.exp(3 *X1)/(torch.exp(3*X1) + torch.exp(2*X1))
z1 = torch.exp(y)
z2 = torch.cos(y)
z3 = torch.sin(y)
z4 = y * y
z5 = torch.exp(3 *y)/(torch.exp(3*y) + torch.exp(2*y))

In [9]:
print(torch.autograd.grad(z1, X1,retain_graph=True))
print(torch.autograd.grad(z2, X1,retain_graph=True))
print(torch.autograd.grad(z3, X1,retain_graph=True))
print(torch.autograd.grad(z4, X1,retain_graph=True))
print(torch.autograd.grad(z5, X1,retain_graph=True))

(tensor(0.1171),)
(tensor(-0.0368),)
(tensor(0.0262),)
(tensor(0.0861),)
(tensor(0.0091),)


100個のランダムな3次元のfloat型Tensorを持つDatasetクラスを自作してください
- 条件
  - lenが100
  - 同じindexを指定した時に同じ値が帰ってく

In [8]:
from  torch.utils.data.dataset import Dataset

In [16]:
class RandomDataset(Dataset):
    def __init__(self):
        self.dataset = torch.rand(100,3)
    
    def __len__(self):
        return 100
    
    def __getitem__(self, index):
        return self.dataset[index]

rd = RandomDataset()
print(len(rd))
print(rd[0])
print(rd[0])

100
tensor([0.1564, 0.8517, 0.5447])
tensor([0.1564, 0.8517, 0.5447])


## 演習
入力4次元,出力3次元の活性化関数がReLUの三層のDNNをnn.Sequential, nn.Module二通りの使い方で実装しなさい

In [4]:
import torch.nn.functional as F

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(4, 3)
        self.l2 = nn.Linear(3, 3)
        
    def forward(self, x):
        o = self.l1(x)
        o = F.relu(o)
        return self.l2(o)
        
net = Net()
net(torch.tensor([1., 2., 3., 4.]))

tensor([ 0.4754,  0.7621, -0.1280], grad_fn=<AddBackward0>)

In [2]:
net = nn.Sequential(nn.Linear(4,3), nn.ReLU(), nn.Linear(3, 3))
net(torch.tensor([1., 2., 3., 4.]))

tensor([ 0.5681, -0.3191,  0.5662], grad_fn=<AddBackward0>)

## 演習
以下のソースコードはそのままでは動かない.何箇所か修正し,活性化関数ReLUの3層のnnの計算を実行せよ.
```python
class Net(Module):
    def __init__(self, netwoerks):
        self.networks = networks

    def forward(self, x):
        for net in self.networks:
            net(x)
net = Net([nn.Linear(3,5), nn.Linear(5, 4)])
net(torch.tensor([1, 2, 3]))
```

## 回答は自分が作ってください

## 演習
以下のソースコードではそのままでは動かない.何箇所が修正し,(1,2,3,4)に対する出力が1になるようなSoftmax回帰を実装せよ.
```python
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
    
    def forward(self, x):
        l1 = nn.Linear(4, 3)
        return l1(x)
    
net, x, y = Net(), torch.tensor([[1,2,3,4]]), torch.tensor([1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters())
loss = net(x)
[loss.backward() for _ in range(100)]    
```

## 回答は自分が作ってください

## 演習
以下のソースコードではそのままでは動かない.何箇所が修正し,(1,2,3,4)に対する出力が1になるようなNNを実装せよ.
```python
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.nn_list = [nn.Linear(4,4), nn.Linear(4, 3)]
    
    def forward(self, x):
        for l in nn_list:
            x = nn.ReLU()(l(x))
        return x,
net, x, y = Net(), torch.tensor([1,2,3,4], [2,3,10, 1]), torch.tensor([1, 2])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters())
for i in range(100):
    criterion(net(x), y).backward(); optimizer.step()
```

## 回答は自分で作ってください